## Part one
Today's problem is incredibly well-suited to array programming. Let's start by assuming the data is already parsed in the shape we want (the ⍝ (lamp) just indicates a line comment):

In [6]:
⍝ 2-4,6-8
⍝ 2-3,4-5
⍝ 5-7,7-9
⍝ 2-8,3-7
⍝ 6-6,4-6
⍝ 2-6,4-8
start1←2 2 5 2 6 2
end1←4 3 7 8 6 6
start2←6 4 7 3 4 4
end2←8 5 9 7 6 8

All of APL's comparison functions are *rank-polymorphic*, meaning that when they are applied to arrays, they compare elements in those arrays pairwise. APL has no booleans, so all comparison functions return binary arrays. ∧ means "and" and ∨ means "or".

In [7]:
1<2
1 2<2 1
0 1∧1 1

1

1 0

0 1

The solution to part one is just simple logic:

In [8]:
⊢x←((start1≥start2)∧(end1≤end2))∨((start1≤start2)∧(end1≥end2))
+/x

0 0 0 1 1 0

2

## Parsing
Now the question is, how do we get the input into that shape?

Let's start by pulling out the numbers and ditching the rest of the characters. Dyadic ∊ is "member of", so combined with ⎕D (a string of the numbers 0-9) we can get a binary array indicating where the numbers are.

In [9]:
input←'2-4,6-8' '2-3,4-5' '5-7,7-9' '2-8,3-7' '6-6,4-6' '2-6,4-8'
⊢line←⊃input
line∊⎕D

2-4,6-8

1 0 1 0 1 0 1

Now we want to split that array up. Dyadic ⊆ (partition) is perfect for this, it groups contiguous spans of 1s into subarrays and removes 0s.

In [10]:
1 1 0 1 1⊆1 2 3 4 5

┌───┬───┐
│1 2│4 5│
└───┴───┘

⊆ is designed specifically to work well with grouping based on conditions: you apply a condition that puts 0s where you want splits to be, then you use ⊆ to make the splits.

In [11]:
(line∊⎕D)⊆line

┌─┬─┬─┬─┐
│2│4│6│8│
└─┴─┴─┴─┘

Now we just need to apply that over the whole input

In [12]:
{(⍵∊⎕D)⊆⍵}¨input

┌─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│
││2│4│6│8│││2│3│4│5│││5│7│7│9│││2│8│3│7│││6│6│4│6│││2│6│4│8││
│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│
└─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

But wait, this is a great opportunity to use a *fork*! Remember, a fork allows you to write `(f x) h (g x)` as `(f g h)x`. We have an ⍵ on both sides of that dfn, and they're getting combined with ⊆. On the left our function is ∊⎕D, and on the right, we can use ⊢ (identity) to return the input unchanged.

One problem though: ∊ wants our line to be its *left* argument, but when we use ¨ (each) to map over the input it'll be provided on the *right*. To fix that, We can use "swap" (⍨) to flip ∊'s arguments.

Once we do that though, we need a way to provide ⎕D as ∊'s right argument. ∘ (bind) allows us to turn a dyadic function into a monadic one by partially-applying one of the arguments. With that, we have:

In [13]:
(⎕D∘(∊⍨)⊆⊢)¨input

┌─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│
││2│4│6│8│││2│3│4│5│││5│7│7│9│││2│8│3│7│││6│6│4│6│││2│6│4│8││
│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│
└─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

Why the parentheses around ∊⍨? Look how APL parses the code without them:

In [14]:
(⎕D∘(∊⍨)⊆⊢)
(⎕D∘∊⍨⊆⊢)

┌─┼─┐
 ∘ ⊆ ⊢
┌┴─┐
⎕D ⍨
 ┌─┘
 ∊

┌─┼─┐
 ⍨ ⊆ ⊢
 ┌─┘
 ∘
┌┴─┐
⎕D ∊

⍨ gets applied to the entirety of ⎕D∘∊ instead of just ∊, which would put ⎕D on ∊'s *right*, not *left*.

I'm not sure why (∊∘⎕D⍨⊆⊢) doesn't work, if you figure it out let me know!

One final diversion on forks: we could also write this like so:

In [15]:
⎕D(∊⍨⊆⊢)line

┌─┬─┬─┬─┐
│2│4│6│8│
└─┴─┴─┴─┘

That's right, forks can take both left *and* right arguments! The full definition of a fork is:
```
a(f g h)b = (a f b) h (a g b)
```
There's also a dual to ⊢: ⊣ is the identity function of its *left* argument.

Though to map it over the input, we have to reintroduce ∘.

In [16]:
⎕D∘(∊⍨⊆⊢)¨input

┌─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│┌─┬─┬─┬─┐│
││2│4│6│8│││2│3│4│5│││5│7│7│9│││2│8│3│7│││6│6│4│6│││2│6│4│8││
│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│└─┴─┴─┴─┘│
└─────────┴─────────┴─────────┴─────────┴─────────┴─────────┘

Setting aside playing with forks, now we want to transform this nested array of arrays of single numbers into an array with all the first elements, an array with all the second elements, etc. APL has the ⍉ (transpose) function, but it works on matricies, not nested arrays. That's easy to fix though, we can just ↑ (mix) the array into a matrix, transpose, and then ↓ (split) it back into a nested array.

In [17]:
↑(⎕D∘(∊⍨)⊆⊢)¨input
⍉↑(⎕D∘(∊⍨)⊆⊢)¨input
↓⍉↑(⎕D∘(∊⍨)⊆⊢)¨input

┌─┬─┬─┬─┐
│2│4│6│8│
├─┼─┼─┼─┤
│2│3│4│5│
├─┼─┼─┼─┤
│5│7│7│9│
├─┼─┼─┼─┤
│2│8│3│7│
├─┼─┼─┼─┤
│6│6│4│6│
├─┼─┼─┼─┤
│2│6│4│8│
└─┴─┴─┴─┘

┌─┬─┬─┬─┬─┬─┐
│2│2│5│2│6│2│
├─┼─┼─┼─┼─┼─┤
│4│3│7│8│6│6│
├─┼─┼─┼─┼─┼─┤
│6│4│7│3│4│4│
├─┼─┼─┼─┼─┼─┤
│8│5│9│7│6│8│
└─┴─┴─┴─┴─┴─┘

┌─────────────┬─────────────┬─────────────┬─────────────┐
│┌─┬─┬─┬─┬─┬─┐│┌─┬─┬─┬─┬─┬─┐│┌─┬─┬─┬─┬─┬─┐│┌─┬─┬─┬─┬─┬─┐│
││2│2│5│2│6│2│││4│3│7│8│6│6│││6│4│7│3│4│4│││8│5│9│7│6│8││
│└─┴─┴─┴─┴─┴─┘│└─┴─┴─┴─┴─┴─┘│└─┴─┴─┴─┴─┴─┘│└─┴─┴─┴─┴─┴─┘│
└─────────────┴─────────────┴─────────────┴─────────────┘

At some point, we should also ⍎ (execute) the string digits into numbers.

In [18]:
↓⍉↑(⍎¨⎕D∘(∊⍨)⊆⊢)¨input

┌───────────┬───────────┬───────────┬───────────┐
│2 2 5 2 6 2│4 3 7 8 6 6│6 4 7 3 4 4│8 5 9 7 6 8│
└───────────┴───────────┴───────────┴───────────┘

Now we just have to jam the parsing together with the solution, and part one is complete!

In [19]:
parsed←↓⍉↑(⍎¨⎕D∘(∊⍨)⊆⊢)¨input
start1 end1 start2 end2 ← parsed
+/((start1≥start2)∧(end1≤end2))∨((start1≤start2)∧(end1≥end2))

2

## Part two
The parsing is the same for part two, the solution just uses slightly different logic. Rather than looking for fully-contained ranges, we're now looking for any overlapping ranges. I found it easier to test for when ranges *don't* overlap, and then negate the result using ~.

In [20]:
(end1<start2)∨(end2<start1)
~(end1<start2)∨(end2<start1)
+/~(end1<start2)∨(end2<start1)

1 1 0 0 0 0

0 0 1 1 1 1

4

And that's it for part two!